DIMENSION TABLES TEAMS, PLAYERS, VENUES & DATES

In [0]:
DROP TABLE dw.dim_team CASCADE;

CREATE TABLE IF NOT EXISTS dw.dim_team (
  team_id   BIGINT IDENTITY(1,1) PRIMARY KEY,
  team_key  VARCHAR(256) UNIQUE NOT NULL,  -- normalized for joins
  team_name VARCHAR(128) NOT NULL,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
DISTSTYLE AUTO;


DROP TABLE dw.dim_player CASCADE;

CREATE TABLE IF NOT EXISTS dw.dim_player (
  player_id   BIGINT IDENTITY(1,1) PRIMARY KEY,      -- surrogate key
  player_key  VARCHAR(512) UNIQUE NOT NULL,          -- player_name + team_key
  full_name   VARCHAR(256) NOT NULL,
  team_id     BIGINT NOT NULL REFERENCES dw.dim_team(team_id),
  created_at  TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
DISTSTYLE AUTO;

DROP TABLE dw.dim_venue CASCADE;

CREATE TABLE IF NOT EXISTS dw.dim_venue (
  venue_id    BIGINT IDENTITY(1,1) PRIMARY KEY,
  venue_key   VARCHAR(512) UNIQUE NOT NULL,        -- normalized "venue|city|country"
  venue_name  VARCHAR(256),
  city        VARCHAR(128),
  country     VARCHAR(64)
)
DISTSTYLE AUTO;

DROP TABLE dw.dim_date CASCADE;

CREATE TABLE IF NOT EXISTS dw.dim_date (
  date_key   INT PRIMARY KEY,                      -- yyyymmdd
  full_date  DATE UNIQUE NOT NULL,
  year       INT,
  month      INT,
  day        INT,
  week       INT,
  quarter    INT,
  season     VARCHAR(16)
)
DISTSTYLE AUTO;

CORE MATCH DIMENSION TABLE

In [0]:
DROP TABLE dw.dim_match CASCADE;

CREATE TABLE IF NOT EXISTS dw.dim_match (
  match_id         BIGINT PRIMARY KEY,            -- = stg.match_table.match_type_number
  event_name       VARCHAR(256),
  match_type       VARCHAR(64),
  match_stage      VARCHAR(64),
  date_key         INT NOT NULL REFERENCES dw.dim_date(date_key),
  event_date       DATE NOT NULL,
  season           VARCHAR(64),
  team_type        VARCHAR(32),
  overs            INT,
  venue_id         BIGINT REFERENCES dw.dim_venue(venue_id),
  gender           VARCHAR(16),
  first_team_id    BIGINT REFERENCES dw.dim_team(team_id),
  second_team_id   BIGINT REFERENCES dw.dim_team(team_id),
  match_result     VARCHAR(64),
  winner_team_id   BIGINT REFERENCES dw.dim_team(team_id),
  toss_winner_id   BIGINT REFERENCES dw.dim_team(team_id),
  toss_decision    VARCHAR(16),
  stg_file_name    VARCHAR(512),
  stg_file_hashkey VARCHAR(256),
  ingest_date      DATE
)
DISTKEY(match_id)
SORTKEY(event_date, match_id);